# Analysis Notebook for the Paper "Data-driven exploration and continuum modeling of dislocation networks"

By running this notebook, you should be able to reproduce all results and plots from Section 2 of the paper "Data-driven exploration and continuum modeling of dislocation networks".
(Provided you have the data and you have adapted the hard-coded path to the CSV file).
The overall goal is to predict reaction densities (*glissile*, *lomer*, *hirth*) based on dislocation densities.

In [ ]:
import math
import matplotlib.pyplot as plt # plotting
from matplotlib import cm # color mapping
import numpy as np # numeric arrays
import pandas as pd # data handling
import re # regular expressions
import scipy as sp # descriptive stats
import seaborn as sns # grouped boxplots
from sklearn import linear_model, preprocessing # ML

## Load data

Apart from column type conversion and the removal of an id column, there is no other pre-processing here.

In [ ]:
path = 'data/delta_sampled_merged_last_voxel_data_size2400_order2_speedUp2.csv'
df = pd.read_csv(path)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')] # exclude unnamed columns
df = df.apply(lambda x: pd.to_numeric(x, errors = 'raise')) # convert column types

### Compute (exploration/prediction) targets

We are interested in reaction densities for three reaction types.
The reaction densities are summed over the 12 slip systems.

In [ ]:
targetFeatures = ['rho_glissile', 'rho_lomer', 'rho_coll']
for targetFeature in targetFeatures:
    df[targetFeature] = (df[targetFeature + '_1'])
    for gs in range(2,13): # sum over slip systems
        df[targetFeature] = df[targetFeature] + (df[targetFeature + '_' + str(gs)])

## Explore and plot data (Section 2.1 and 2.2)

Before making predictions, we explore the data set.

### General dataset characteristics (Section 2.1)

We begin by making some general statements about the size of the dataset:

In [ ]:
print('Overall dataset size:', df.shape)
for targetFeature in targetFeatures:
    print('Actual number of data objects for ' + targetFeature + ':', sum(df[targetFeature] != 0))
print('Voxel layout: ' + str(len(df.pos_x.unique())) + '*' + str(len(df.pos_y.unique())) + '*' + str(len(df.pos_z.unique())))
print(str(len(df.time.unique())) + ' time steps: ' + ', '.join([str(x) for x in sorted(df.time.unique())]))

### Histograms of target features

We plot histograms combined with Gaussian kernel density estimates.

In [ ]:
for targetFeature in targetFeatures:
    fig, ax = plt.subplots(nrows = 1, ncols = 2)
    fig.set_size_inches(w = 15, h = 3)
    sns.distplot(df[targetFeature], ax = ax[0])
    sns.distplot(df[targetFeature], ax = ax[1])
    ax[1].set_yscale('log')

### Target features over time

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 3)
fig.set_size_inches(w = 15, h = 3)
for i in range(len(targetFeatures)):
    sns.regplot(x = 'time', y = targetFeatures[i], data = df, ax = ax[i])
plt.tight_layout() # prevents overlap of subfigures

### Target features over space

In [ ]:
for targetFeature in targetFeatures:
    fig, ax = plt.subplots(nrows = 1, ncols = 3)
    fig.set_size_inches(w = 15, h = 3)
    sns.boxplot(x = 'pos_x', y = targetFeature, data = df, ax = ax[0]) # can also use scatterplot
    sns.boxplot(x = 'pos_y', y = targetFeature, data = df, ax = ax[1])
    sns.boxplot(x = 'pos_z', y = targetFeature, data = df, ax = ax[2])
    plt.tight_layout() # prevents overlap of subfigures

### Dislocation density vs reaction density (Section 2.2, Figure 2 a/b/c)

We provide some insights into the ground truth by plotting reaction density vs. dislocation density.
This also motivates predictions with a linear model.

In [ ]:
plt.rcParams.update({'font.size': 15})
a = 0.4040496e-9
volume = 5e-6*5e-6*5e-6
plotFeatures = ['rho_glissile', 'rho_lomer', 'rho_coll']
yLabels = [r'Glissile reaction density $\rho_\mathrm{gliss}~[$m$^{-2}]$',\
           r'Lomer reaction density $\rho_\mathrm{lomer}~[$m$^{-2}]$',\
           r'Collinear reaction density $\rho_\mathrm{coll}~[$m$^{-2}]$']
for i in range(len(plotFeatures)):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_xlim([1e11, 1e13])
    ax.set_ylim([1e7,1e11])
    ax.set_xscale('log')
    ax.set_yscale('log')  
    plotData = df[df[plotFeatures[i]] != 0]  
    plt.scatter(plotData['rho_tot'] * a / volume, plotData[plotFeatures[i]] * a / volume,\
                c = plotData['n_loops'], cmap = cm.viridis, marker = 'o', vmin = 0, vmax = 600)
    cb = plt.colorbar(label = r'Number of dislocations $[-]$')
    plt.grid(linestyle = ':')
    plt.xlabel(r'Total dislocation density $\rho_\mathrm{tot}~[$m$^{-2}]$')
    plt.ylabel(yLabels[i])
    plt.savefig('plots/' + plotFeatures[i].replace('rho_', '') + '_ground_truth_loglog_numberloops.pdf',
                bbox_inches = "tight")

## Analyze and predict data (Section 2.3)

Now we want to desscribe the relationship between dislocation density and reaction density with prediction models.
We will try out different feature sets, corresponding to the equations in the paper.

### Define reaction pairs

These pairs of slip systems are relevant for reactions, based on domain knowledge.
When we compute interaction features later, we will only consider these pre-defined pairs instead of all combinations of slip systems.

In [ ]:
reactionPairs = {
    'rho_coll': [[1,12], [2,6], [3,9], [4,10], [5,8], [7,11]],
    'rho_lomer': [[1,4], [1,7], [2,8], [2,10], [3,5], [3,11], [4,7], [5,11], [6,9], [6,12], [8,10], [9,12]],
    'rho_glissile': [[1,6], [1,9], [1,10], [1,11], [2,4], [2,5], [2,9], [2,12], [3,6], [3,7],\
                     [3,8], [3,12], [4,8], [4,11], [4,12], [5,7], [5,9], [5,10], [6,8], [6,10],\
                     [7,10], [7,12], [8,11], [9,11]]
}

### Feature Engineering

We compute three types of features, separately for each reaction type.
All features are based on the pre-defined reaction types and combine dislocation densities from different slip systems:

- single interaction terms, i.e., two terms for each reaction pair (`system1 * sqrt(system2)` and `sqrt(system1) * system2`)
- biviariate interaction terms, i.e., one term for each reaction pair (sum of the two *single* terms)
- overall interaction term, i.e., sum over all *single* interaction terms

For another analysis, we also keep the raw dislocation densities in the dataset.
Furthermore, we retain the space and time attributes which will come handy when splitting the data in cross-validation.
All irrelevant columns are discarded.
We store thre pre-defined lists of features and will re-use them during prediction.

In [ ]:
densityFeatures = [x for x in list(df) if re.match( '^rho_[0-9]+$', x)] # raw densities; same for all reaction types
reactFeaturesSingle = dict() # specific to reaction type (depends on reaction pairs)
reactFeaturesBi = dict() # specific to reaction type (depends on reaction pairs)
reactFeaturesAll = ['reactAll'] # same name for all reaction types (though computation depends on reaction pairs)
predictionData = dict() # specific to reaction type (depends on features included)

for targetFeature in targetFeatures:
    # Reduce to dislocation densities, split-relevant features, and target
    curPredictionData = df[densityFeatures + ['pos_x', 'pos_y', 'pos_z', 'time', targetFeature]]
    
    # Remove voxels without density
    curPredictionData = curPredictionData[curPredictionData[targetFeature] != 0]
    
    # Engineer interaction features
    curPredictionData['reactAll'] = np.zeros(curPredictionData.shape[0])
    for pair in reactionPairs[targetFeature]:
        name1 = 'reactSingle_' + str(pair[0]) + '_sqrt' + str(pair[1])
        name2 = 'reactSingle_sqrt' + str(pair[0]) + "_" + str(pair[1])
        value1 = curPredictionData['rho_' + str(pair[0])] *\
            curPredictionData['rho_' + str(pair[1])].apply(lambda x: math.sqrt(x))
        value2 =  curPredictionData['rho_' + str(pair[1])] *\
            curPredictionData['rho_' + str(pair[0])].apply(lambda x: math.sqrt(x))
        curPredictionData[name1] = value1
        curPredictionData[name2] = value2
        curPredictionData['reactBi_' + str(pair[0]) + '_' + str(pair[1])] = value1 + value2
        curPredictionData['reactAll'] = curPredictionData['reactAll'] + value1 + value2
    reactFeaturesSingle[targetFeature] = [x for x in list(curPredictionData) if re.match( '^reactSingle', x)]
    reactFeaturesBi[targetFeature] = [x for x in list(curPredictionData) if re.match( '^reactBi', x)]
    predictionData[targetFeature] = curPredictionData

In [ ]:
# # Also add single interaction terms for interactions not in pre-defined interaction pairs
# reactFeaturesNot = dict() # specific to reaction type (depends on reaction pairs)
# for targetFeature in targetFeatures:
#     for i in range(1, 12):
#         for j in range(i+1, 13):
#             if [i, j] not in reactionPairs[targetFeature]:
#                 name1 = 'reactNot_' + str(i) + '_sqrt' + str(j)
#                 name2 = 'reactNot_sqrt' + str(i) + "_" + str(j)
#                 value1 = predictionData[targetFeature]['rho_' + str(i)] *\
#                     predictionData[targetFeature]['rho_' + str(j)].apply(lambda x: math.sqrt(x))
#                 value2 =  predictionData[targetFeature]['rho_' + str(j)] *\
#                     predictionData[targetFeature]['rho_' + str(i)].apply(lambda x: math.sqrt(x))
#                 predictionData[targetFeature][name1] = value1
#                 predictionData[targetFeature][name2] = value2
#     reactFeaturesNot[targetFeature] = [x for x in list(curPredictionData) if re.match( '^reactNot', x)]

#### Correlation of different feature types with summed reaction density (Figure 4)

Correlate the different feature sets with the standard target, which is the reaction density summed over all slip systems.


In [ ]:
plt.rcParams.update({'font.size': 15})
plotFeatures = ['rho_glissile', 'rho_lomer', 'rho_coll']
plotTitles = ['Glissile reaction', 'Lomer reaction', 'Collinear reaction']
fig, ax = plt.subplots(nrows = 1, ncols = 3, sharey = True)
fig.set_size_inches(w = 15, h = 5)
for i in range(len(plotFeatures)):
    plotFeature = plotFeatures[i]
    inputData = predictionData[plotFeature]
    inputData = inputData.drop(columns = ['pos_x', 'pos_y', 'pos_z', 'time'])
    correlations = [np.corrcoef(inputData[feature], inputData[plotFeature])[0,1] for feature in list(inputData)]
    corrData = pd.DataFrame({'Feature': list(inputData), 'Correlation': correlations, 'Category': ''})
    corrData.loc[corrData.Feature.isin(densityFeatures), 'Category'] = 'Dislocation densities'
    corrData.loc[corrData.Feature.isin(reactFeaturesSingle[plotFeature]), 'Category'] = 'Eq. (1)'
    corrData.loc[corrData.Feature.isin(reactFeaturesBi[plotFeature]), 'Category'] = \
        r'Eq. (1) with $\forall\xi~\forall\zeta~\beta_1^{\xi\zeta} = \beta_2^{\xi\zeta}$'
    corrData.loc[corrData.Feature.isin(reactFeaturesAll), 'Category'] = 'Eq. (2)'
#     corrData.loc[corrData.Feature.isin(reactFeaturesNot[plotFeature]), 'Category'] = 'ReactNot'
#     corrData.loc[corrData.Feature == plotFeature, 'Category'] = 'Target'
    corrData = corrData[corrData.Feature != plotFeature]
    sns.boxplot(y = 'Category', x = 'Correlation', data = corrData, ax = ax[i])
    ax[i].set_title(plotTitles[i])
    ax[i].set_ylabel('Feature set')
plt.tight_layout()
plt.savefig('plots/feature_correlation.pdf')

#### Correlation of different feature types with reaction density on slip systems

Correlate the different feature sets with the individual reaction densities on the different slip systems.
The box plot then summarizes this over all slip systems and all features of a certain type.

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 3, sharey = True)
fig.set_size_inches(w = 15, h = 5)
for i in range(len(targetFeatures)):
    targetFeature = targetFeatures[i]
    inputData = predictionData[targetFeature]
    inputData = inputData.drop(columns = ['pos_x', 'pos_y', 'pos_z', 'time'])
    correlations = [np.corrcoef(inputData[feature],
                                df.loc[df[targetFeature] != 0, targetFeature + '_' + str(targetNumber)])[0,1]
                    for feature in list(inputData) for targetNumber in range(1,13)]
    corrData = pd.DataFrame({'Feature': np.repeat(list(inputData), 12), 'Correlation': correlations, 'Category': ''})
    corrData.loc[corrData.Feature.isin(densityFeatures), 'Category'] = 'RawDensity'
    corrData.loc[corrData.Feature.isin(reactFeaturesSingle[targetFeature]), 'Category'] = 'ReactSingle'
    corrData.loc[corrData.Feature.isin(reactFeaturesBi[targetFeature]), 'Category'] = 'ReactBi'
    corrData.loc[corrData.Feature.isin(reactFeaturesAll), 'Category'] = 'ReactAll'
#     corrData.loc[corrData.Feature.isin(reactFeaturesNot[targetFeature]), 'Category'] = 'ReactNot'
    corrData.loc[corrData.Feature == targetFeature, 'Category'] = 'Target'
    sns.boxplot(y = 'Category', x = 'Correlation', data = corrData, ax = ax[i])
    ax[i].set_title(targetFeature)
    ax[i].set_ylabel('Feature category')

### Prediction pipeline

Our prediction pipeline is rather short.
It consists of:

- (temporal) train-test split
- dropping highly correlated features
- min-max scaling
- (optional) filter feature selection based on Pearson correlation
- training of a linear regression model
- prediction
- evaluation
- (optional) predicted-vs-ground-truth plot and various diagnostic plots

In [ ]:
# Finds highly correlated columns in the train data and removes them from train as well as test data.
# https://stackoverflow.com/a/44674459
def dropCorrelatedFeatures(X_train, X_test = None):
    corr_cols = []
    corr_matrix = X_train.corr().abs()
    for i in range(len(corr_matrix.columns)):
        if (corr_matrix.columns[i] not in corr_cols):
            for j in range(i):
                if (corr_matrix.iloc[i, j] >= 0.95) and (corr_matrix.columns[j] not in corr_cols):
                    corr_cols.append(corr_matrix.columns[i])
    X_train = X_train.drop(columns = corr_cols)
    if X_test is not None:
        X_test = X_test.drop(columns = corr_cols)
    return X_train, X_test

# Trains a scaling approach on the train data and returns the scaled train data and test data.
def scaleData(X_train, X_test = None):
    scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))
    scaler = scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train), columns = X_train.columns)
    if X_test is not None:
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)
    return X_train, X_test

# Creates a plot with predicted vs ground truth values and saves it to the hard-drive.
def plotPredictedVsGroundTruth(predicted, groundTruth, targetName):
    plt.rcParams.update({'font.size': 15})
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(groundTruth, predicted, 'ro', markersize = 5) # scatter plot with red circles
    mxY=max(max(groundTruth), max(predicted))
    mnY=min(min(groundTruth), min(predicted))
    plt.plot([mnY,mxY], [mnY,mxY], 'k') # k means color = black
    plt.xlabel('ground truth value')
    plt.ylabel('predicted value')
    plt.tight_layout()
    plt.grid(linestyle=':')
    plt.savefig('plots/' + targetName.replace('rho_', '') + '_predicted_vs_ground_truth_scaling.pdf')

# Creates various diagnostic plots.
def plotDiagnostics(predicted, groundTruth):
    residuals = predicted - groundTruth
    fig, ax = plt.subplots(nrows = 1, ncols = 4)
    fig.set_size_inches(w = 15, h = 4)
    sns.distplot(a = residuals, ax = ax[0], axlabel = 'Residuals')
    ax[0].set_title('Residuals distribution')
    sp.stats.probplot(x = residuals, dist = 'norm', plot = ax[1])
    ax[1].set_title('Residuals Q-Q')
    sns.scatterplot(x = predicted, y = residuals, ax = ax[2])
    ax[2].set_xlabel('Predicted')
    ax[2].set_ylabel('Residuals')
    ax[2].set_title('Residuals vs. predicted')
    sns.scatterplot(x = groundTruth, y = residuals, ax = ax[3])
    ax[3].set_xlabel('Ground truth')
    ax[3].set_ylabel('Residuals')
    ax[3].set_title('Residuals vs. ground truth')
    plt.tight_layout() # prevents overlap of subfigures

# Runs the full prediction pipeline and returns the trained regression model,
# training set scores and test set scores.
# Predicted-vs-ground-truth plots and filter feature selection are optional.
# If you want to use the latter, you can specify the number of features to be
# selected (absolute or fraction).
def evaluateWithLM(dataset, features, target, plot = False, filterFS = 0):

    # Train-test split
    X_train = dataset[features][dataset.time <= 6000]
    X_test = dataset[features][dataset.time > 6000]
    y_train = dataset[target][dataset.time <= 6000]
    y_test = dataset[target][dataset.time > 6000]

    # Drop highly correlated features
    X_train, X_test = dropCorrelatedFeatures(X_train, X_test)

    # Scaling
    X_train, X_test = scaleData(X_train, X_test)

    # Filter feature selection (optional)
    if filterFS > 0:
        if filterFS < 1: # relative number of features
            filterFS = round(filterFS * len(features)) # turn absolute
        if filterFS < X_train.shape[1]: # else feature selection does not make sense
            corrCoeffs = [sp.stats.pearsonr(X_train[x], y_train)[0] for x in list(X_train)]
            topFeatureIndices = np.argsort([-abs(x) for x in corrCoeffs])[0:filterFS] # sort abs correlation decreasingly
            topFeatures = [list(X_train)[x] for x in topFeatureIndices]
            X_train = X_train[topFeatures]
            X_test = X_test[topFeatures]

    # Training
    lm = linear_model.LinearRegression()
    reg = lm.fit(X_train, y_train)

    # Prediction
    y_pred = reg.predict(X_train)
    y_test_pred = reg.predict(X_test)

    # Evaluation
    print('Train R^2:', round(reg.score(X_train, y_train), 3))
    print('Test R^2:', round(reg.score(X_test, y_test), 3))
    print('Summary of coefficients:', sp.stats.describe(reg.coef_))
    
    # Plotting (optional)
    if plot:
        plotPredictedVsGroundTruth(predicted = y_test_pred, groundTruth = y_test, targetName = target)
        plotDiagnostics(predicted = y_test_pred, groundTruth = y_test)

    return reg

### Base model (two interaction terms per reaction pair, Figure 3 a/b/c)

We evaluate the quality of a linear regression model for the reaction density.
Each interaction term gets its own coefficient in the model.
We also create several diagnostic plots.

In [ ]:
# Analyzes the difference between two regression coefficients belonging to same reaction pair.
# Only works if there are two features for each reaction pair.
def evaluateCoefficientDiff(regModel, features, plot = True):
    # Assume each two (odd, even) elements in "features" belong to the same reaction pair
    featureNamesWithoutSqrt = [x.replace('sqrt', '') for x in features]
    assert featureNamesWithoutSqrt[::2] == featureNamesWithoutSqrt[1::2] # odd == even?
    assert len(regModel.coef_) == len(features)
    coefficients_1_2sqrt = regModel.coef_[::2]
    coefficients_sqrt1_2 = regModel.coef_[1::2]
    coefficientDiff = coefficients_1_2sqrt - coefficients_sqrt1_2
    relCoefficientDiff = abs(coefficientDiff / coefficients_1_2sqrt * 100)
    print('Deviation in % of 2nd to 1st coefficient within reaction pair:', sp.stats.describe(relCoefficientDiff))
    if plot:
        fig, ax = plt.subplots(nrows = 1, ncols = 2)
        fig.set_size_inches(w = 15, h = 3)
        ax[0].boxplot(relCoefficientDiff, showfliers = False) # excludes outliers
        ax[0].set_xlabel('Deviation in % of 2nd to 1st coefficient')
        ax[1].hist(relCoefficientDiff, range = (0,200), bins = 20)
        ax[1].xaxis.set_major_locator(plt.MultipleLocator(20)) # ticks
        ax[1].set_xlabel('Deviation in % of 2nd to 1st coefficient')

for targetFeature in targetFeatures:
    print('----- ' + targetFeature + ' -----')
    regModel = evaluateWithLM(dataset = predictionData[targetFeature],\
        features = reactFeaturesSingle[targetFeature], target = targetFeature, plot = True)
    plt.show()

### Filter feature selection

First, we correlate the Pearson correlation of each (interaction) feature with the prediction target.
Next, we summarize and plot this correlation.
Finally, we train a regression model using just a fraction of the features, selecting them by absolute correlation.
Note that the top features used there might differ from the top features found in the explorative analysis presented first, because the filter feature selection for prediction is only conducted on the training data (as it should be) and not the full dataset.

In [ ]:
for targetFeature in targetFeatures:
    print('----- ' + targetFeature + ' -----')
    corrCoeffs = [sp.stats.pearsonr(predictionData[targetFeature][x],\
        predictionData[targetFeature][targetFeature])[0] for x in reactFeaturesSingle[targetFeature]]
    print('\nSummary of correlation of interaction features with target:', sp.stats.describe(corrCoeffs))
    fig, ax = plt.subplots(nrows = 1, ncols = 2)
    fig.set_size_inches(w = 15, h = 3)
    ax[0].boxplot(corrCoeffs)
    ax[0].set_xlabel('Correlation of features with target')
    ax[1].hist(corrCoeffs, range = (0,1), bins = 20)
    ax[1].xaxis.set_major_locator(plt.MultipleLocator(0.1)) # ticks
    ax[1].set_xlabel('Correlation of features with target')
    plt.show()
    topFeatureIndices = np.argsort([-abs(x) for x in corrCoeffs])[0:5] # sort absolute correlation decreasingly
    topFeatures = [reactFeaturesSingle[targetFeature][x] for x in topFeatureIndices]
    print('Top 5 highest-correlated features:', topFeatures)
    print('Model with 1/3 highest-correlated features:')
    evaluateWithLM(dataset = predictionData[targetFeature], features = reactFeaturesSingle[targetFeature],\
                   target = targetFeature, plot = False, filterFS = 1/3)
    print('Model with 1/6 highest-correlated features:')
    evaluateWithLM(dataset = predictionData[targetFeature], features = reactFeaturesSingle[targetFeature],\
                   target = targetFeature, plot = False, filterFS = 1/6)

### Bivariate interaction terms (one interaction term per reaction pair)

This model merges the two coefficients of each reaction pair.

In [ ]:
for targetFeature in targetFeatures:
    print('----- ' + targetFeature + ' -----')
    evaluateWithLM(dataset = predictionData[targetFeature], features = reactFeaturesBi[targetFeature],\
                   target = targetFeature, plot = False)

### One (overall) interaction term

This model only has one coefficient, which weights the sum over all reactions.

In [ ]:
for targetFeature in targetFeatures:
    print('----- ' + targetFeature + ' -----')
    evaluateWithLM(dataset = predictionData[targetFeature], features = reactFeaturesAll,\
                   target = targetFeature, plot = False)

### No interaction terms

This model only uses the dislocation densities from the 12 slip systems.
No interaction terms between slip systems are considered.

In [ ]:
for targetFeature in targetFeatures:
    print('----- ' + targetFeature + ' -----')
    evaluateWithLM(dataset = predictionData[targetFeature], features = densityFeatures,\
                   target = targetFeature, plot = False)